<div>
     <div>
        <img src="./report/isel_logo.png" width="400" height="400" align="left">
    </div>
    <div>
        <h2>Área Departamental de Engenharia de Eletrónica e Telecomunicações e de Computadores</h2>
        <p>Trabalho prático 4</p>
        <p>Autor:	44598	André L. A. Q. de Oliveira</p>
        <p>Unidade Curricular Compressão de Sinais Multimédia</p>
        <p>Professor: André Lourenço</p>
        <p>27 - Junho - 2021</p>
    </div>
</div>

### <a id="index"></a>

# Index
- [Tabelas da norma ITU T.81 - JPEG standard](#ITU_T.81)
- [Compressão](#compressao)
    - [snr](#snr)
- [Entropia](#entropia)
    - [entropy](#entropy)
    - [efficiency](#efficiency)
- [Utilities](#utilities)
    - [imshow](#imshow)
    - [convertFrame2Jpeg](#convertFrame2Jpeg)
    - [createPframe](#createPframe)
    - [mae](#mae)
    - [divide_to_blocks](#divide_to_blocks)
    - [merge_from_blocks](#merge_from_blocks)
- [Codificador 1](#codificador_1)
    - [encode_1](#encode_1)
- [Codificador 2](#codificador_2)
    - [encode_2](#encode_2)
    - [decode_2](#decode_2)
- [Codificador 3](#codificador_2)
    - [encode_3](#encode_3)
    - [decode_3](#decode_3)
- [Testes](#testes)
- [Conclusões](#conclusoes)

# Importar bibliotecas

In [276]:
import os
import math
from time import time
import cv2
import numpy as np
import matplotlib.pyplot as plt

cwd = os.getcwd() # current work diretory

<a id="compressao"></a>

# Compressão

A taxa de compressão pode ser cáculada atrás da seguinte expressão matemática:

$$ T_c = \frac{D_o}{D_c} $$

<a id="snr"></a>

## SNR

A relação sinal-ruído compara o nível de um sinal desejado com o nível do ruído. Quanto mais alta for a relação sinal-ruído, menor é o efeito do ruído de fundo sobre a detecção ou medição do sinal.

$$ SNR(Db) = 10\log_{10}\left[\frac{\sum_{l}\sum_{c}I_{ap}(l,c)^2}{\sum_{l}\sum_{c}[I_{ap}(l,c)-I_{or}(l,c)]^2}\right] $$

In [278]:
def snr(I_or, I_ap):
    Pxa = np.sum(I_ap.astype('float')**2)
    Pe = np.sum( (I_ap.astype('float') - I_or.astype('float'))**2 )
    return 10 * np.log10(Pxa / Pe)

<a id="entropia"></a>

# Entropia

A entropia mede a quantidade de informação codificada na mensagem, onde quando maior for o valor entrópico, maior será a incerteza.

A entropia da fonte é dada pela seguinte expressão matemática:

$$ H(S) = -\sum_{i=1}^{N}p(s_i)log_{2}p(s_i) $$


A eficiência da condificação pode ser obtida através da seguinte expressão matemática:

$$ \eta = \frac{H(S)}{L} $$

onde, **_L_** é o número médio de bits por símbolo.

É possível codificar uma fonte, sem perdas, se o número médio de bits por símbolo for maior ou igual à entropia da fonte:

$$ H(S) < L < H(S) + \delta $$

<a id="entropy"></a>

## entropy


In [279]:
def calculate_entropy(symbol_freq_dictionary):
    # list of symbol occurences in file
    occurrences = list(symbol_freq_dictionary.values())
    # total number of symbols
    t = np.sum(occurrences)
    # probability of each symbol
    p = [occ / t for occ in occurrences]
    return -np.sum([p * np.log2(p)])

<a id="efficiency"></a>

## efficiency

In [280]:
def efficiency(H, dictionary):
    # average number of bits per symbol
    L = 0
    for symbol in dictionary:
        L += len(dictionary[symbol])
    L = L / len(dictionary)
    return L, ( H / L )

[back to index](#index)

<a id="utilities"></a>

# Utilities

<a id="imshow"></a>

## imshow

In [281]:
def imshow(title, image):
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

<a id="convertFrame2Jpeg"></a>

## convertFrame2Jpeg

In [282]:
def convertFrame2Jpeg(path, frame):
    cv2.imwrite(f"{path}.jpg", frame, (cv2.IMWRITE_JPEG_QUALITY, 50))
    return cv2.imread(f"{path}.jpg", cv2.IMREAD_GRAYSCALE)

<a id="create_Pframe"></a>

## create_Pframe

In [283]:
def create_Pframe(Iframe, frame):
    return ( ( Iframe.astype('int16') - frame.astype('int16') ) / 2  + 127 ).astype('uint8')

<a id="reconstruct_Iframe"></a>

## reconstruct_Iframe

In [286]:
def reconstruct_Iframe(Iframe, Pframe):
    return ( Iframe.astype('int16') - (Pframe.astype('int16') - 127)*2).astype('uint8')

In [309]:
i_f = frames[0]
print("i_f\n",i_f)

f = frames[1]
print("f\n",f)

p_f = create_Pframe(i_f, f)
print("p_f\n",p_f)

r_f = reconstruct_Iframe(i_f,p_f)
print("r_f\n", r_f)

i_f
 [[ 60  62  62 ...  30  32  32]
 [ 68  69  75 ...  31  33  32]
 [ 89  57  58 ...  30  33  32]
 ...
 [ 62  63  62 ...  62  55  28]
 [ 62  64  65 ... 120 119  99]
 [ 65  63  64 ... 119 119 119]]
f
 [[ 59  60  65 ...  29  31  32]
 [ 69  71  72 ...  29  32  32]
 [ 98  58  55 ...  29  32  33]
 ...
 [ 62  63  63 ...  69  59  36]
 [ 63  63  62 ... 120 121 107]
 [ 64  64  64 ... 119 119 119]]
p_f
 [[127 128 125 ... 127 127 127]
 [126 126 128 ... 128 127 127]
 [122 126 128 ... 127 127 126]
 ...
 [127 127 126 ... 123 125 123]
 [126 127 128 ... 127 126 123]
 [127 126 127 ... 127 127 127]]
r_f
 [[ 60  60  66 ...  30  32  32]
 [ 70  71  73 ...  29  33  32]
 [ 99  59  56 ...  30  33  34]
 ...
 [ 62  63  64 ...  70  59  36]
 [ 64  64  63 ... 120 121 107]
 [ 65  65  64 ... 119 119 119]]


<a id="mae"></a>

## Erro absoluto médio (MAE)

Critério de semelhança entre blocos

$$ (d_m,d_n) = arg \min_{d_m,d_n} \frac{1}{MN}\sum_{m=1}^{M}\sum_{n=1}^{N} \left| X^{t}(m,n)-X^{t-1}(m-d_m,n-d_n) \right| $$

In [289]:
def mae(b1, b2):
    return np.sum(np.abs(cv2.subtract(b1,b2).ravel())) / len(b1.ravel())

In [290]:
block1 = np.ones(256).reshape((16,16))
block2 = (np.ones(256)*3).reshape((16,16))

print(mae(block1, block2))

2.0


<a id="to_macro_blocks"></a>

### to_macro_blocks

Divide uma imagem em uma matriz de blocos de tamanho size x size.

In [291]:
def to_macro_blocks(image, size):
    
    width, height = image.shape
    
    if (width % size) != 0 or (height % size) != 0:
        raise Exception('function only supports multiple macro block division!')
    
    macro_blocks = np.zeros((int(width/size), int(height/size)), dtype=object)
    
    i = -1
    j = -1
    for lin in range(0, width, size):
        i += 1
        j = -1
        for col in range(0, height, size):
            j += 1
            # create new block
            macro_blocks[i][j] = image[lin:(lin + size),col:(col  +size)]
                
    return macro_blocks

<a id="from_macro_blocks"></a>

### from_macro_blocks

Merge um matriz de macro blocos para uma imagem.

In [292]:
def from_macro_blocks(matrix, size):
    
    width  = matrix.shape[0] * size
    heigth = matrix.shape[1] * size
    
    output = np.zeros((width, heigth), dtype='uint8')
    
    i = -1
    j = -1
    for lin in range(0, width, size):
        i += 1
        j = -1
        for col in range(0, heigth, size):
            j += 1
            output[lin:(lin+size),col:(col+size)] = matrix[i][j]
    return output

In [310]:
f = np.arange(64).reshape((8,8))
print('f\n', frame)

mBlocks = to_macro_blocks(f, 4)
print('divite to blocks\n', mBlocks)

r_f = from_macro_blocks(mBlocks,4)
print('merge from blocks\n', r_f)

f
 [[ 0  1  2  3  4  5  6  7]
 [ 8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23]
 [24 25 26 27 28 29 30 31]
 [32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47]
 [48 49 50 51 52 53 54 55]
 [56 57 58 59 60 61 62 63]]
divite to blocks
 [[array([[ 0,  1,  2,  3],
         [ 8,  9, 10, 11],
         [16, 17, 18, 19],
         [24, 25, 26, 27]]) array([[ 4,  5,  6,  7],
                                   [12, 13, 14, 15],
                                   [20, 21, 22, 23],
                                   [28, 29, 30, 31]])]
 [array([[32, 33, 34, 35],
         [40, 41, 42, 43],
         [48, 49, 50, 51],
         [56, 57, 58, 59]]) array([[36, 37, 38, 39],
                                   [44, 45, 46, 47],
                                   [52, 53, 54, 55],
                                   [60, 61, 62, 63]])]]
merge from blocks
 [[ 0  1  2  3  4  5  6  7]
 [ 8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23]
 [24 25 26 27 28 29 30 31]
 [32 33 34 35 36 37 38 39]
 [40 41 42 43 44 

 <a id="codificador_1"></a>
 
# Codificador 1
 
<div class="alert alert-block alert-info">
Considera que cada frame é uma intra-frame (I).
</div>

In [295]:
def encode_1(path, frames):
    Iframes = []
    for i in range(len(frames)):
        Iframes.append( convertFrame2Jpeg(f"{path}/Iframe_{i+1}", frames[i]) )
    return Iframes

<a id="codificador_2"></a>
 
# Codificador 2
 
<div class="alert alert-block alert-info">
Considera que todas as frames à exceção da primeira (a I-frame) são inter-frames (P), sem compensação de movimento.
</div>

In [311]:
def encode_2(path, frames):
    Iframe = frames[0]
    Pframes = []
    convertFrame2Jpeg(f"{path}/Iframe_{1}", Iframe)
    for i in range(1, len(frames)):
        Pframes.append( convertFrame2Jpeg(f"{path}/Pframe_{i+1}", create_Pframe(Iframe, frames[i])) )
    return Iframe, Pframes

In [297]:
def decode_2(path, Iframe, Pframes):
    rIframes = []
    for i in range(len(Pframes)):
        rIframes.append( convertFrame2Jpeg(f"{path}/rIframe_{i+2}", reconstruct_Iframe(Iframe, Pframes[i])) )
    return Iframe, rIframes

<a id="codificador_3"></a>
 
# Codificador 3
 
<div class="alert alert-block alert-info">
Considera que todas as frames à exceção da primeira (a I-frame) são inter-frames (P), com compensação de movimento.
</div>

In [301]:
def encode_3(path, frames):
    
    # block size
    bs = 16
    # window size
    ws = 15
    
    # first frame it's Iframe
    Iframe = frames[0]
    frames = frames[1:]
    
    # Pframes
    Pframes = []
    
    for frame in frames:
        # divide frame i into 16x16 macro blocks
        macroBlocks = to_macro_blocks(frame, bs)
        # create output
        output = to_macro_blocks(np.zeros(Iframe.shape, dtype='uint8'), bs)
        s = ""
        print(macroBlocks.shape)
        for i in range(macroBlocks.shape[0]):
            for j in range(macroBlocks.shape[1]):
                ### bounderies
                # top
                if i == 0:
                    # left corner
                    if j == 0:
                        # output[i,j] = window_best_match(Iframe, macroBlocks[i,j], i, j, i+ws, j+ws)
                        s += f"sob - ({i},{j}) "
                    # right corner
                    elif j == (macroBlocks.shape[1] - 1):
                        # output[i,j] = window_best_match(Iframe, macroBlocks[i,j], i, j-ws, i, j)
                        s += f"({i},{j})\n"
                    # center
                    else:
                        # output[i,j] = window_best_match(Iframe, macroBlocks[i,j], i-ws, j-ws, i+ws, j+ws)
                        s += f"({i},{j}) "
                # bottom
                elif i == (macroBlocks.shape[0] - 1):
                    # left corner
                    if j == 0:
                        # output[i,j] = window_best_match(Iframe, macroBlocks[i,j], i-ws, j, i, j+ws)
                        s += f"({i},{j}) "
                    # right corner
                    elif j == (macroBlocks.shape[1] - 1):
                        # output[i,j] = window_best_match(Iframe, macroBlocks[i,j], i-ws, j-ws, i, j)
                        s += f"({i},{j}) - eob"
                    # center
                    else:
                        # output[i,j] = window_best_match(Iframe, macroBlocks[i,j], i-ws, j-ws, i, j+ws)
                        s += f"({i},{j}) "
                # left
                elif j == 0:
                    # output[i,j] = window_best_match(Iframe, macroBlocks[i,j], i-ws, j, i+ws, j+ws)
                    s += f"({i},{j}) "
                # right
                elif j == (macroBlocks.shape[1] - 1):
                    # output[i,j] = window_best_match(Iframe, macroBlocks[i,j], i-ws, j-ws, i+ws, j)
                    s += f"({i},{j})\n"
                # center
                else:
                    # output[i,j] = window_best_match(Iframe, macroBlocks[i,j], i-ws, j-ws, i+ws, j+ws)
                    s += f"({i},{j}) "
        print(s)
        # Pframes.append( convertFrame2Jpeg(f"{path}/Pframe_{i+1}", create_Pframe(Iframe, output)) )
      
    return Iframe, Pframes

In [300]:
def sortDictionaryByDistance(d: dict,x: int, y: int):
    return dict(sorted(d.items(), key=lambda item: math.sqrt((x-item[0][0])**2+(y-item[0][1])**2)))

In [303]:
d = dict()
d[(0,0)] = 3
d[(0,1)] = 7
d[(0,4)] = 6
d[(0,3)] = 2
d[(1,1)] = 2

print(d)
d = sortDictionaryByDistance(d, 1, 3)
print(d)

best_key = min(d, key=d.get)
print(best_key)

{(0, 0): 3, (0, 1): 7, (0, 4): 6, (0, 3): 2, (1, 1): 2}
{(0, 3): 2, (0, 4): 6, (1, 1): 2, (0, 1): 7, (0, 0): 3}
(0, 3)


In [387]:
def window_best_match(Iframe, macroBlock, x, y, x1, y1, x2, y2):
    d = dict()
    for i in range(x1,x2+1): # until
        for j in range(y1,y2+1):
            print(f"i{i}{j}\n", Iframe[i:(i+macroBlock.shape[0]), j:(j+macroBlock.shape[1])])
            d[i,j] = mae(Iframe[i:(i+macroBlock.shape[0]), j:(j+macroBlock.shape[1])], macroBlock)
            
    print(d)
    dByDistance = sortDictionaryByDistance(d, int((x2-x1)/2), int((y2-y1)/2))
    print(dByDistance)
    best_key = min(dByDistance, key=d.get)
    print(best_key)
    return Iframe[best_key[0]:(best_key[0]+macroBlock.shape[0]), best_key[1]:(best_key[1]+macroBlock.shape[1])]     

In [388]:
if3 = [[ 0, 0, 0, 0],
       [ 0, 0, 0, 0],
       [ 1, 1, 1, 0],
       [ 1, 1, 1, 0],
       [ 1, 1, 1, 0],
       [ 0, 0, 0, 0]]
if3 = np.array(if3)

i3 = [[ 0, 0, 0, 0],
      [ 0, 0, 0, 0],
      [ 0, 0, 0, 0],
      [ 1, 1, 1, 0],
      [ 1, 1, 1, 0],
      [ 1, 1, 1, 0]]
i3 = np.array(i3)

mb = to_macro_blocks(i3, 2)
print(mb)

best_candidate = window_best_match(if3, mb[1,1], 1, 1, 2-2, 2-2, 2+2, 2)
print(best_candidate)

[[array([[0, 0],
         [0, 0]]) array([[0, 0],
                         [0, 0]])]
 [array([[0, 0],
         [1, 1]]) array([[0, 0],
                         [1, 0]])]
 [array([[1, 1],
         [1, 1]]) array([[1, 0],
                         [1, 0]])]]
i00
 [[0 0]
 [0 0]]
i01
 [[0 0]
 [0 0]]
i02
 [[0 0]
 [0 0]]
i10
 [[0 0]
 [1 1]]
i11
 [[0 0]
 [1 1]]
i12
 [[0 0]
 [1 0]]
i20
 [[1 1]
 [1 1]]
i21
 [[1 1]
 [1 1]]
i22
 [[1 0]
 [1 0]]
i30
 [[1 1]
 [1 1]]
i31
 [[1 1]
 [1 1]]
i32
 [[1 0]
 [1 0]]
i40
 [[1 1]
 [0 0]]
i41
 [[1 1]
 [0 0]]
i42
 [[1 0]
 [0 0]]
{(0, 0): 0.25, (0, 1): 0.25, (0, 2): 0.25, (1, 0): 0.25, (1, 1): 0.25, (1, 2): 0.0, (2, 0): 0.75, (2, 1): 0.75, (2, 2): 0.25, (3, 0): 0.75, (3, 1): 0.75, (3, 2): 0.25, (4, 0): 0.75, (4, 1): 0.75, (4, 2): 0.5}
{(2, 1): 0.75, (1, 1): 0.25, (2, 0): 0.75, (2, 2): 0.25, (3, 1): 0.75, (1, 0): 0.25, (1, 2): 0.0, (3, 0): 0.75, (3, 2): 0.25, (0, 1): 0.25, (4, 1): 0.75, (0, 0): 0.25, (0, 2): 0.25, (4, 0): 0.75, (4, 2): 0.5}
(1, 2)
[[0 0]
 [1 0]]


In [364]:
def decode_3(path, frames):

SyntaxError: unexpected EOF while parsing (<ipython-input-364-e8f03d7105e7>, line 1)

### <a id="testes"></a>

# Testes

In [ ]:
error() # cause erros to stop here

## Importar dados

In [ ]:
# sorting frames names by alphabetical order
# frames[0] = sorted(frames[0], key=lambda x: (x[0],int(x[5:])) )

In [ ]:
frames = []
for i in range(len(os.listdir(f"{cwd}/input_data/bola_seq"))):
    frames.append( cv2.imread(f"{cwd}/input_data/bola_seq/bola_{i+1}.tiff", cv2.IMREAD_GRAYSCALE) )

<div class="alert alert-block alert-info">
Codificador 1
</div>

In [ ]:
Iframe1 = encode_1(f"{cwd}/output_data/codificador1", frames)

<div class="alert alert-block alert-info">
Codificador 2
</div>

In [ ]:
Iframe2, Pframes2 = encode_2(f"{cwd}/output_data/codificador2/encode", frames)

In [ ]:
Iframes2, rIframes2 = decode_2(f"{cwd}/output_data/codificador2/decode", Iframe2, Pframes2)

<div class="alert alert-block alert-info">
Codificador 3
</div>

[back to index](#index)

<a id="conclusoes"></a>

# Conclusões

Este trabalho explora os princípios básicos da codificação de vídeo. Neste trabalho pretende-se implementar três formas
de codificação de vídeo. Cada um destes codificadores deve ser testado com a a sequência de imagens disponibilizadas

[back to index](#index)